In [28]:
# Imports
import os
import pandas as pd

# Configuration
name = 'TL4_gDNA_ADBC_ge5_LUT' #UPDATE the name you want associated with all of your files
input_dir = 'adbc_lut_summaries'  #Update with folder containing all of the summary files you want to combine
count_column = 'Read Count'      #UPDATE Column to rename in each file with the basefile name
merge_key = 'Category'                      #UPDATE Column to merge on
output_dir = 'Merged_Summary_Tables'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [29]:
#Merge all summary tables 
# Collect processed DataFrames
merged_tables = []

# Loop through CSV files
for file in os.listdir(input_dir):
    if file.endswith('.csv'):
        file_path = os.path.join(input_dir, file)
        
        # Extract sample name (base name up to first '.')
        sample_name = file.split('.')[0]
        
        # Read and rename count column
        df = pd.read_csv(file_path)
        df_renamed = df.rename(columns={count_column: f'{sample_name}_ReadCount'})
        
        # Append to list
        merged_tables.append(df_renamed)

# Merge all tables on the merge key
final_summary = merged_tables[0]
for df in merged_tables[1:]:
    final_summary = pd.merge(final_summary, df, on=merge_key, how='outer')

# Display preview
final_summary.head()


,Category,TL4_gDNA_ADBC_a1_gDNA_ADBC_S1_ReadCount,TL4_gDNA_ADBC_a2_gDNA_ADBC_S2_ReadCount,TL4_gDNA_ADBC_a3_gDNA_ADBC_S3_ReadCount,TL4_gDNA_ADBC_a4_gDNA_ADBC_S4_ReadCount,TL4_gDNA_ADBC_a5_gDNA_ADBC_S5_ReadCount,TL4_gDNA_ADBC_a6_gDNA_ADBC_S6_ReadCount,TL4_gDNA_ADBC_a7_gDNA_ADBC_S7_ReadCount,TL4_gDNA_ADBC_a8_gDNA_ADBC_S8_ReadCount
0,Map3 Starting Read Count,877604,978658,1460997,899400,785044,813678,1287393,882002
1,Initial Unique HawkBCs,2509,2685,3448,2600,2202,2433,3118,2531
2,Starting Unique HawkBC + ADBC2 Count,27759,28768,32687,29796,15758,22175,26144,28711
3,Map3 >= 5 reads per HA (HawkBC + ADBC2),844484,944193,1420760,863311,767921,788270,1256478,847738
4,Final Unique HawkBCs (after filtering),251,254,275,251,245,250,274,256


**Below tracks % remaining after each filtering step**

There are different code blocks for different number of counts you want to keep track of and if you want different formatting for having the % column next to is respective count column of all percents at the end after all the counts columns

In [33]:
# Configuration: three totals and their associated rows
totals_config = {                         #UPDATE with the correct column names total_row is the row you want to be used as the inital count to base percentages on, rows are the rows you want to take the percent remaining 
    'ReadCount': {
        'total_row': 'Map3 Starting Read Count',
        'rows': [
            'Map3 Starting Read Count',
            'Map3 >= 5 reads per HA (HawkBC + ADBC2)'
        ]
    },
    'HawkBCs': {
        'total_row': 'Initial Unique HawkBCs',
        'rows': [
            'Initial Unique HawkBCs',
            'Final Unique HawkBCs (after filtering)'
        ]
    },
    'HA': {
        'total_row': 'Starting Unique HawkBC + ADBC2 Count',
        'rows': [
            'Starting Unique HawkBC + ADBC2 Count',
            'Map3 >= 5 Unique HA'
        ]
    }
}

# Identify sample columns
sample_columns = [col for col in final_summary.columns if col.endswith('_ReadCount')]

# Track new column order
new_column_order = ['Category']

# Convert all sample columns to numeric
for col in sample_columns:
    final_summary[col] = final_summary[col].apply(
        lambda x: pd.to_numeric(str(x).replace(',', ''), errors='coerce')
    )

# Create one %_Total column per sample
for col in sample_columns:
    # Format read count with commas
    final_summary[col] = final_summary[col].apply(
        lambda x: f"{int(x):,}" if pd.notnull(x) else None
    )
    
    # Initialize %_Total column
    percent_col = col.replace('_ReadCount', '_%Total')
    final_summary[percent_col] = None
    
    # Apply each config to its respective rows
    for label, config in totals_config.items():
        total_row_label = config['total_row']
        rows_to_calculate = config['rows']
        
        # Get total value
        total_row = final_summary.loc[final_summary['Category'] == total_row_label, col]
        if total_row.empty:
            raise ValueError(f"Total row '{total_row_label}' not found for column '{col}'")
        total_value = int(str(total_row.values[0]).replace(',', ''))
        
        # Calculate and assign % values
        for idx, row in final_summary.iterrows():
            if row['Category'] in rows_to_calculate and pd.notnull(row[col]):
                val = int(str(row[col]).replace(',', ''))
                pct = round((val / total_value) * 100, 1)
                final_summary.at[idx, percent_col] = pct

    # Add columns to final order
    new_column_order.append(col)
    new_column_order.append(percent_col)

# Reorder columns
final_summary = final_summary[new_column_order]

In [30]:
# # Configuration format of all the Readcount first then all % at the end
# total_row_label = 'Total Num of sequences'  # Label identifying the total row
# rows_to_calculate = [                        # Rows to calculate % of total for
#     'Total Num of sequences',
#     'Total Correct RTBC len'
# ]

# # Process each sample column
# sample_columns = [col for col in final_summary.columns if col.endswith('_ReadCount')]

# for col in sample_columns:
#     # Convert to numeric (remove commas if present)
#     final_summary[col] = final_summary[col].apply(
#         lambda x: pd.to_numeric(str(x).replace(',', ''), errors='coerce')
#     )
    
#     # Create corresponding %_Total column
#     percent_col = col.replace('_ReadCount', '_%Total')
    
#     # Extract total value for this sample
#     total_value = final_summary.loc[final_summary['Category'] == total_row_label, col].values[0]
    
#     # Calculate % of total for specified rows
#     final_summary[percent_col] = final_summary.apply(
#         lambda row: round((row[col] / total_value) * 100, 1)
#         if row['Category'] in rows_to_calculate and pd.notnull(row[col]) else None,
#         axis=1
#     )
    
#     # Format read count with commas
#     final_summary[col] = final_summary[col].apply(
#         lambda x: f"{int(x):,}" if pd.notnull(x) else None
#     )

# # Preview updated table
# final_summary.head()



In [31]:
# # Configuration of ReadCount next to % for the same sample
# total_row_label = 'Map3 Starting Read Count'
# rows_to_calculate = [
#     'Map3 Starting Read Count',
#     'Map3 >= 5 reads per RTBC'
# ]

# # Identify sample columns
# sample_columns = [col for col in final_summary.columns if col.endswith('_ReadCount')]

# # Track new column order
# new_column_order = ['Category']

# for col in sample_columns:
#     # Convert to numeric for calculation
#     final_summary[col] = final_summary[col].apply(
#         lambda x: pd.to_numeric(str(x).replace(',', ''), errors='coerce')
#     )
    
#     # Extract total value
#     total_row = final_summary.loc[final_summary['Category'] == total_row_label, col]
#     if total_row.empty:
#         raise ValueError(f"Total row '{total_row_label}' not found for column '{col}'")
#     total_value = total_row.values[0]
    
#     # Calculate % of total
#     percent_values = final_summary.apply(
#         lambda row: round((row[col] / total_value) * 100, 1)
#         if row['Category'] in rows_to_calculate and pd.notnull(row[col]) else None,
#         axis=1
#     )
    
#     # Format read count with commas
#     final_summary[col] = final_summary[col].apply(
#         lambda x: f"{int(x):,}" if pd.notnull(x) else None
#     )
    
#     # Create % column
#     percent_col = col.replace('_ReadCount', '_%Total')
#     final_summary[percent_col] = percent_values
    
#     # Append both to new column order
#     new_column_order.append(col)
#     new_column_order.append(percent_col)

# # Reorder columns
# final_summary = final_summary[new_column_order]

# # Preview
# final_summary.head()


In [32]:
# #when there are multiple 2 totals to calculate (read % , unique count rtbc) 
# # Configuration: two total types with their rows
# totals_config = {
#     'ReadCount': {
#         'total_row': 'Map3 Starting Read Count',
#         'rows': [
#             'Map3 Starting Read Count',
#             'Map3 >= 5 reads per RTBC'
#         ]
#     },
#     'UniqueCount': {
#         'total_row': 'Starting Unique RTBC Count',
#         'rows': [
#             'Starting Unique RTBC Count',
#             'Map3 >= 5 Unique RTBC'
#         ]
#     }
# }

# # Identify sample columns
# sample_columns = [col for col in final_summary.columns if col.endswith('_ReadCount')]

# # Track new column order
# new_column_order = ['Category']

# # Convert all sample columns to numeric
# for col in sample_columns:
#     final_summary[col] = final_summary[col].apply(
#         lambda x: pd.to_numeric(str(x).replace(',', ''), errors='coerce')
#     )

# # Create one %_Total column per sample
# for col in sample_columns:
#     # Format read count with commas
#     final_summary[col] = final_summary[col].apply(
#         lambda x: f"{int(x):,}" if pd.notnull(x) else None
#     )
    
#     # Initialize %_Total column with None
#     percent_col = col.replace('_ReadCount', '_%Total')
#     final_summary[percent_col] = None
    
#     # Apply each config to its respective rows
#     for label, config in totals_config.items():
#         total_row_label = config['total_row']
#         rows_to_calculate = config['rows']
        
#         # Get total value
#         total_row = final_summary.loc[final_summary['Category'] == total_row_label, col]
#         if total_row.empty:
#             raise ValueError(f"Total row '{total_row_label}' not found for column '{col}'")
#         total_value = int(str(total_row.values[0]).replace(',', ''))
        
#         # Calculate and assign % values
#         for idx, row in final_summary.iterrows():
#             if row['Category'] in rows_to_calculate and pd.notnull(row[col]):
#                 val = int(str(row[col]).replace(',', ''))
#                 pct = round((val / total_value) * 100, 1)
#                 final_summary.at[idx, percent_col] = pct

#     # Add columns to final order
#     new_column_order.append(col)
#     new_column_order.append(percent_col)

# # Reorder columns
# final_summary = final_summary[new_column_order]
# final_summary

In [34]:
# Save merged summary csv 
output_path = os.path.join(output_dir, f'{name}_Merged_Summary_Table.csv')
final_summary.to_csv(output_path, index=False)

# Save merged summary as Excel
excel_output_path = os.path.join(output_dir, f'{name}_Merged_Summary_Table.xlsx')
final_summary.to_excel(excel_output_path, index=False)

In [35]:
final_summary

,Category,TL4_gDNA_ADBC_a1_gDNA_ADBC_S1_ReadCount,TL4_gDNA_ADBC_a1_gDNA_ADBC_S1_%Total,TL4_gDNA_ADBC_a2_gDNA_ADBC_S2_ReadCount,TL4_gDNA_ADBC_a2_gDNA_ADBC_S2_%Total,TL4_gDNA_ADBC_a3_gDNA_ADBC_S3_ReadCount,TL4_gDNA_ADBC_a3_gDNA_ADBC_S3_%Total,TL4_gDNA_ADBC_a4_gDNA_ADBC_S4_ReadCount,TL4_gDNA_ADBC_a4_gDNA_ADBC_S4_%Total,TL4_gDNA_ADBC_a5_gDNA_ADBC_S5_ReadCount,TL4_gDNA_ADBC_a5_gDNA_ADBC_S5_%Total,TL4_gDNA_ADBC_a6_gDNA_ADBC_S6_ReadCount,TL4_gDNA_ADBC_a6_gDNA_ADBC_S6_%Total,TL4_gDNA_ADBC_a7_gDNA_ADBC_S7_ReadCount,TL4_gDNA_ADBC_a7_gDNA_ADBC_S7_%Total,TL4_gDNA_ADBC_a8_gDNA_ADBC_S8_ReadCount,TL4_gDNA_ADBC_a8_gDNA_ADBC_S8_%Total
0,Map3 Starting Read Count,"877,604",100.0,"978,658",100.0,"1,460,997",100.0,"899,400",100.0,"785,044",100.0,"813,678",100.0,"1,287,393",100.0,"882,002",100.0
1,Initial Unique HawkBCs,"2,509",100.0,"2,685",100.0,"3,448",100.0,"2,600",100.0,"2,202",100.0,"2,433",100.0,"3,118",100.0,"2,531",100.0
2,Starting Unique HawkBC + ADBC2 Count,"27,759",100.0,"28,768",100.0,"32,687",100.0,"29,796",100.0,"15,758",100.0,"22,175",100.0,"26,144",100.0,"28,711",100.0
3,Map3 >= 5 reads per HA (HawkBC + ADBC2),"844,484",96.2,"944,193",96.5,"1,420,760",97.2,"863,311",96.0,"767,921",97.8,"788,270",96.9,"1,256,478",97.6,"847,738",96.1
4,Final Unique HawkBCs (after filtering),251,10.0,254,9.5,275,8.0,251,9.7,245,11.1,250,10.3,274,8.8,256,10.1
5,Map3 >= 5 Unique HA,"2,266",8.2,"2,308",8.0,"2,718",8.3,"2,397",8.0,"1,931",12.3,"2,061",9.3,"2,248",8.6,"2,339",8.1


**Below color codes the different count rows in the excel file so it is easier to read**

In [37]:
#excel coloring with 3 totals 
from openpyxl import load_workbook
from openpyxl.styles import Font


# Load workbook and sheet
wb = load_workbook(excel_output_path)
ws = wb.active

# Get row labels from config
ha_rows = totals_config['HA']['rows']
hawkbc_rows = totals_config['HawkBCs']['rows']

# Apply font color based on category
for row in ws.iter_rows(min_row=2):  # Skip header
    category = row[0].value
    if category in ha_rows:
        for cell in row:
            cell.font = Font(color="0000FF")  # Blue
    elif category in hawkbc_rows:
        for cell in row:
            cell.font = Font(color="008000")  # Green

# Save styled workbook
wb.save(excel_output_path)


In [36]:
# #color code excel file with 2 total counts
# from openpyxl import load_workbook
# from openpyxl.styles import Font

# # Load workbook and active sheet
# wb = load_workbook(excel_output_path)
# ws = wb.active

# # Rows to color (from UniqueCount config)
# unique_rows = [
#     'Starting Unique RTBC Count',
#     'Map3 >= 5 Unique RTBC'
# ]

# # Apply blue font to matching rows
# for row in ws.iter_rows(min_row=2):  # Skip header
#     category = row[0].value
#     if category in unique_rows:
#         for cell in row:
#             cell.font = Font(color="0000FF")  # Blue text

# # Save updated workbook
# wb.save(excel_output_path)
